In [1]:
import os
os.chdir('..')

In [2]:
from ddi_kt_2024.mol.gnn import GNN
from ddi_kt_2024.utils import load_pkl
from ddi_kt_2024.mol.preprocess import mapped_smiles_reader, candidate_smiles
from ddi_kt_2024.mol.mol_dataset import MolDataset

2024-03-11 22:55:07,874 - INFO - Enabling RDKit 2023.09.5 jupyter extensions


In [3]:
all_candidates_train = load_pkl('cache/pkl/v1/candidates.train.pkl')
all_candidates_test = load_pkl('cache/pkl/v1/candidates.test.pkl')
mapped_smiles = mapped_smiles_reader('cache/mapped_drugs/DDI/all_mapped.txt')
x_train, y_train = candidate_smiles(all_candidates_train, mapped_smiles)
x_test, y_test = candidate_smiles(all_candidates_test, mapped_smiles)
# dataset_train_mol1 = MolDataset(x_train, element=1)
# dataset_train_mol2 = MolDataset(x_train, element=2)
dataset_test_mol1 = MolDataset(x_test, element=1)
dataset_test_mol2 = MolDataset(x_test, element=2)

Converting SMILES to PyG: 100%|██████████| 5716/5716 [00:08<00:00, 667.57it/s] 


In [4]:
from torch_geometric.loader import DataLoader
test_loader_mol1 = DataLoader(dataset_test_mol1, batch_size=128, shuffle=False)
test_loader_mol2 = DataLoader(dataset_test_mol2, batch_size=128, shuffle=False)

In [8]:
model = GNN(gnn_option='ATTENTIVEFP', readout_option='global_max_pool')

In [9]:
for x in test_loader_mol1:
    print(model(x).shape)
    break

torch.Size([128, 128])
